# Train and compare configurations

In [1]:
from medigraph.train_multiple_models import train_multiple_configurations as train
from medigraph.model.metrics import plot_metrics
from medigraph import root_dir
from pathlib import Path
import torch
%load_ext autoreload
%autoreload 2

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
n_epochs = 1000
features_selection = ["rfe",]
models_list = ["Dense", "Single-h=16"]
output_folder = root_dir/"results"
assert output_folder.exists(), "Output folder does not exist"

In [ ]:
metric_dict = train(
    models_list=models_list,
    device=device,
    n_epochs=n_epochs,
    features_selection_list=features_selection,
    output_folder=output_folder
)

In [ ]:
plot_metrics(metric_dict)